<a href="https://colab.research.google.com/github/WetSuiteLeiden/data-collection/blob/master/api_tweede_kamer_part2_second_api_verslagen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Purpose of this notebook

Continuing from part 1 -- The SyncFeed API is perfectly functional, 
though it leaves you to do interpretation of the relations yourself, so let's see if the OData API is any more help.


### OData interface

There is a helpful library out there, [tkapi](https://github.com/openkamer/tkapi), which means much less work for us.

In [ ]:
# if you haven't already:
# !pip3 install tkapi

In [1]:
import collections
import wetsuite.helpers.localdata
import wetsuite.helpers.split
from tkapi.document import Document, DocumentSoort
import tkapi, tkapi.document   

In [2]:
# API object we'll be using a few times
api = tkapi.TKApi()

In [3]:
# Get an idea of what this API even does
#   the things that fetch are function calls.

list(name   for name in dir(api)   if name.startswith('get_'))

['get_activiteiten',
 'get_agendapunten',
 'get_all_items',
 'get_antwoorden',
 'get_besluiten',
 'get_commissies',
 'get_documenten',
 'get_dossiers',
 'get_fractie_zetels',
 'get_fracties',
 'get_geschenken',
 'get_item',
 'get_items',
 'get_kamervragen',
 'get_personen',
 'get_reizen',
 'get_related',
 'get_stemmingen',
 'get_vergaderingen',
 'get_verslagen',
 'get_verslagen_van_algemeen_overleg',
 'get_zaken']

### How about verslagen?

Say we want verslagen from commission debates.

Alright, let's look at `get_verslagen()`

In [4]:
all_verslagen = api.get_verslagen( ) # currently ~20K items, will take a few dozen seconds to fetch the metadata of
len(all_verslagen)   

19904

In [6]:
#these are objects:
first_verslag = all_verslagen[0]

first_verslag

In [9]:
# ...so you're probably looking for the parameters on them. 
# The documentation helps a little, but also we can inspect that object:

list( name  for name in dir(all_verslagen[0])   if not name.startswith('_'))

['begin_date_key',
 'create_filter',
 'end_date_key',
 'expand_params',
 'filter_param',
 'get_date_from_datetime_or_none',
 'get_date_or_none',
 'get_datetime_or_none',
 'get_param_expand',
 'get_params_default',
 'get_property_enum_or_none',
 'get_property_or_empty_string',
 'get_property_or_none',
 'get_resource_url_or_none',
 'get_year_or_none',
 'gewijzigd_op',
 'id',
 'orderby_param',
 'print_json',
 'related_item',
 'related_items',
 'related_items_deep',
 'soort',
 'status',
 'type',
 'url',
 'vergadering']

In [12]:
# let's look at the values of some:
first_verslag.gewijzigd_op.date(), first_verslag.soort, first_verslag.status, first_verslag.type, first_verslag.url, first_verslag.vergadering

(datetime.date(2024, 6, 5),
 <VerslagSoort.TUSSENPUBLICATIE: 'Tussenpublicatie'>,
 <VerslagStatus.ONGECORRIGEERD: 'Ongecorrigeerd'>,
 'Verslag',
 'https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Verslag(7c3d4173-6528-4f1b-9dd4-3eba1e0bd322)',
 <tkapi.vergadering.Vergadering at 0x7ff5efe378e0>)

It seems that soort, status, and type are not very helpful.

Also, it becomes clear that we should have read the documentation/model a little better -- these verslagen seem specific to vergaderingen.

How about sorting through _documents_, by type?

In [14]:
# NOTE: YOU MAY WISH TO SKIP THIS and go to the part where we do it properly.
# ..because when you remove that 10000-item limit to count _everthing_, this seems to take half an hour, and easily fails.
all_documenten = api.get_documenten( max_items=10000 ) 

# count soorten
soorten = collections.defaultdict(list)
for document in all_documenten:
    try:
        soorten[ str(document.soort).split(".",1)[1] ].append( document ) # takes the enum name (rather than value) and splits off the DocumentSoort.
    except Exception as e:
        print("SKIP invalid soort: %s"%(e))

# print soort  sorted by count, descending
soorten_by_count = sorted( list( soorten.items() ), key=lambda pair: len(pair[1]), reverse=True )
for soort, doclist in soorten_by_count:
    print( f'{len(doclist):7d}  {soort}')

   2531  BRIEF_REGERING
   1274  BIJLAGE
    871  SCHRIFTELIJKE_VRAGEN
    558  ANTWOORD_SCHRIFTELIJKE_VRAGEN
    545  CONVOCATIE_COMMISSIEACTIVITEIT
    542  BRIEF_COMMISSIE_AAN_BEWINDSPERSOON
    464  MOTIE
    195  VERSLAG_INITIATIEFWETSVOORSTEL_NADER
    183  MEMORIE_VAN_TOELICHTING
    182  VOORSTEL_VAN_WET
    178  STENOGRAM
    156  KONINKLIJKE_BOODSCHAP
    151  MEDEDELING_UITSTEL_ANTWOORD
    121  VERSLAG_VAN_EEN_ALGEMEEN_OVERLEG
    119  AMENDEMENT
    116  NOTA_NAV_HET_NADERTWEEDE_NADERENZ_VERSLAG
    114  BRIEF_LID__FRACTIE
    114  AGENDA_PROCEDUREVERGADERING
    105  NOTA_VAN_WIJZIGING
    102  BESLUITENLIJST_PROCEDUREVERGADERING
     94  ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_NADER_RAPPORT
     89  LIJST_VAN_VRAGEN
     88  MEMORIE_VAN_TOELICHTING_INITIATIEFVOORSTEL
     85  VOORSTEL_VAN_WET_INITIATIEFVOORSTEL
     82  SPREKERSLIJST
     77  LIJST_VAN_VRAGEN_EN_ANTWOORDEN
     72  VERSLAG_VAN_EEN_SCHRIFTELIJK_OVERLEG
     72  BRIEF_COMMISSIE
     71  GELEIDENDE_BRI

Counting is nice to see actual use, but we already can know the right documentsoort without it:
in the middle of the following is `VERSLAG_VAN_EEN_COMMISSIEDEBAT`, which has our example wishes covered.

In [9]:
# list all-uppercase attributes
list(name   for name in dir(DocumentSoort)   if name == name.upper()) 

['AANBIEDINGSBRIEF',
 'AANHANGSEL_VAN_DE_HANDELINGEN',
 'ADVIESAANVRAAG_AFDELING_ADVISERING_RAAD_VAN_STATE',
 'ADVIES_AAN_PRESIDIUM',
 'ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE',
 'ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_NADER_RAPPORT',
 'ADVIES_AFDELING_ADVISERING_RAAD_VAN_STATE_EN_REACTIE_VAN_DE_INITIATIEFNEMERS',
 'ADVIES_COMMISSIE',
 'ADVIES_VAN_ANDERE_ADVIESORGANEN',
 'AGENDA_PLENAIRE_VERGADERING',
 'AGENDA_PROCEDUREVERGADERING',
 'AMENDEMENT',
 'AMENDEMENT_GEWIJZIGD_NADER_VERVANGEND',
 'ANTWOORD_SCHRIFTELIJKE_VRAGEN',
 'ANTWOORD_SCHRIFTELIJKE_VRAGEN_NADER',
 'BEGROTINGSTOELICHTING',
 'BESLUITENLIJST_PROCEDUREVERGADERING',
 'BIJGEWERKTE_TEKST',
 'BIJLAGE',
 'BRIEF_AFDELING_ADVISERING_RAAD_VAN_STATE',
 'BRIEF_ALGEMENE_REKENKAMER',
 'BRIEF_COMMISSIE',
 'BRIEF_COMMISSIE_AAN_BEWINDSPERSOON',
 'BRIEF_CTIVD',
 'BRIEF_EERSTE_KAMER',
 'BRIEF_EUROPESE_COMMISSIE',
 'BRIEF_FORMATEUR',
 'BRIEF_INFORMATEUR',
 'BRIEF_KAMER',
 'BRIEF_LID__FRACTIE',
 'BRIEF_NATIONALE_OMBUDSMAN',
 'BRIEF_PRE

In [6]:
# ...so let's fetch all of that specific documentsoort
# (Note: There is a different take on doing this in part 3, but this one is easier and faster if you only care about the PDFs)

document_filter = Document.create_filter()
document_filter.filter_soort( DocumentSoort.VERSLAG_VAN_EEN_COMMISSIEDEBAT )
commissiedebatverslagen = api.get_documenten( document_filter )

len(commissiedebatverslagen) # 800ish in two seconds, much better

858

In [7]:
commissieverslag_documenten = wetsuite.helpers.localdata.LocalKV('commissieverslag_documenten.db', str, bytes) # store the PDFs in here

for verslag in commissiedebatverslagen:
    print( verslag.titel ) # turns out the titles aren't very indicative of the document content,
    print( verslag.bestand_url )

    wetsuite.helpers.localdata.cached_fetch( commissieverslag_documenten, verslag.bestand_url )



Raad Algemene Zaken en Raad Buitenlandse Zaken
https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(6aad0a05-72e9-4670-9b53-0b3eab523a88)/TK.DA.GGM.OData.Resource()
Raad voor Economische en Financiële Zaken
https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(a64e0e2d-316f-4233-9451-2764baa70305)/TK.DA.GGM.OData.Resource()
Raad Algemene Zaken en Raad Buitenlandse Zaken
https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(a84d9ea7-6a67-45fc-aa80-9e862ea3ea43)/TK.DA.GGM.OData.Resource()
Bestrijding internationaal terrorisme
https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(0dba8510-b96d-4938-9e07-25c74316a607)/TK.DA.GGM.OData.Resource()
Kabinetsaanpak Klimaatbeleid
https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(622f1fe5-2c95-42e0-a009-76aa58b104f8)/TK.DA.GGM.OData.Resource()
Defensieraad
https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(2ad1dfd1-619b-4739-a154-f35a72f08fc0)/TK.DA.GGM.OData.Resource()
Raad Algemene Zaken en

In [8]:
commissieverslag_documenten.summary( get_num_items=True )

{'size_bytes': 123170816,
 'size_readable': '117MiB',
 'num_items': 858,
 'avgsize_bytes': 143556,
 'avgsize_readable': '140KiB'}

In [13]:
# we have a tool to read PDF and extract chunks of text from it. 
# - this will be explained in more detail elsewhere.
# - we can should fine-tune that extractor to, for this case, remove the headers and footers so you can focus on the real text
# - the following code should be made easier to use
# ...but for now, just a preview:

for url, pdfbytes in commissieverslag_documenten.items():
    print( '======= %s ======='%url )
    splitters = wetsuite.helpers.split.decide(pdfbytes, thresh=500)
    for score, fragproc in splitters: # use each processor that said they would be useful
        frags = fragproc.fragments()
        #for _, _, text in frags:
        #    print( text )
        #    print( '--')
        display( wetsuite.helpers.split.SplitDebug(frags) )

    break # just one document for now (all of them is too much for a notebook to show)

-------------- https://gegevensmagazijn.tweedekamer.nl/OData/v4/2.0/Document(00011813-1ac7-48b0-93e6-7fab59345d0f)/TK.DA.GGM.OData.Resource() --------------


meta,intermediate,len,text
{'hints': ['newpage']},{},0,''
"{'hints': ['+para'], 'lastheader': ''}",{},32,'Tweede Kamer der Staten-Generaal'
"{'hints': ['+para'], 'lastheader': ''}",{},2,' 2'
"{'hints': ['+para'], 'lastheader': ''}",{},23,'Vergaderjaar 2020–2021 '
"{'hints': ['bold'], 'lastheader': ''}",{},46,'32 545 Wet- en regelgeving financiële markten '
"{'hints': ['bold'], 'lastheader': ''}",{},65,'Nr. 145 VERSLAG VAN EEN COMMISSIEDEBAT Vastgesteld 28 juli 2021 '
"{'hints': ['bold'], 'lastheader': ''}",{},1821,"'De vaste commissie voor Financiën heeft op 10 juni 2021 overleg gevoerd over: – de brief van de Minister van Financiën d.d. 18 december 2020 inzake gesprek banken over compensatie bij spoofing (Kamer- stuk 32 545, nr. 128); – de brief van de Minister van Financiën d.d. 22 januari 2021 inzake uitkomst aanbesteding giraal betalingsverkeer Rijk (Kamerstuk 26 485, nr. 363); – de brief van de Minister van Financiën d.d. 10 februari 2021 inzake voortgang Actieplan Consumentenkeuzes (Kamerstuk 32 013, nr. 246); – de brief van de Minister van Financiën d.d. 5 februari 2021 inzake antwoorden op vragen commissie over vier BNC-fiches inzake het Digital Finance Package (onder andere Kamerstuk 22112–2949) (Kamerstuk 22 112, nr. 3048); – de brief van de Minister van Financiën d.d. 5 februari 2021 inzake antwoorden op vragen commissie over het fiche: Verordening Markten in Cryptoactiva (MiCA) (Kamerstuk 22112–2937) (Kamerstuk 22 112, nr. 3047); – de brief van de Minister van Financiën d.d. 4 maart 2021 inzake antwoorden op vragen commissie inzake faillissement levensverzekeraar Conservatrix (Kamerstuk 29 507, nr. 153); – de brief van de Minister van Financiën d.d. 11 februari 2021 inzake vervolg ontwikkelingen consumptiefkredietmarkt (Kamerstukken 24 515 en 32 013, nr. 579); – de brief van de Minister van Financiën d.d. 22 februari 2021 inzake kostenvergoeding in verband met de uitspraak Onder- nemingskamer schadeloosstelling onteigening SNS REAAL 2013 (Kamerstuk 33 532, nr. 89); – de brief van de Minister van Financiën d.d. 21 april 2021 inzake wetgevingsbrieven DNB en AFM 2021 (Kamerstuk 32 545, nr. 134); – de brief van de Minister van Financiën d.d. 20 april 2021 inzake reactie op moties en toezeggingen op het terrein van financiële markten voorjaar 2021 (Kamerstuk 32 545, nr. 133); '"
"{'hints': ['+para'], 'lastheader': ''}",{},107,"' kst-32545-145 ISSN 0921 - 7371 ’s-Gravenhage 2021 Tweede Kamer, vergaderjaar 2020–2021, 32 545, nr. 145 1'"
{'hints': ['newpage']},{},0,''
"{'hints': ['+para'], 'lastheader': ''}",{},805,"'de brief van de Minister van Financiën d.d. 12 mei 2021 inzake instelling Evaluatiecommissie Conservatrix (Kamerstuk 29 507, nr. 154); – de brief van de Minister van Financiën d.d. 11 mei 2021 inzake instellen cassatieberoep tegen de beschikking van de Onderne- mingskamer in de schadeloosstellingsprocedure onteigening SNS Reaal (Kamerstuk 33 532, nr. 90); – de brief van de Minister van Financiën d.d. 7 juni 2021 inzake actieve provisietransparantie bij schadeverzekeringen (Kamer- stuk 32 545, nr. 137); – de brief van de Minister van Financiën d.d. 1 juni 2021 inzake eindrapportage rentederivaten Autoriteit Financiële Markten (AFM) (Kamerstuk 31 311, nr. 238); – de brief van de Minister van Financiën d.d. 27 mei 2021 inzake groene obligatie rapportage 2020 (Kamerstuk 35 570 IX, nr. 47). '"
